Data pre-processing

In [1]:
import json
import time
import numpy
'''
Data serialization/de-serialization
Once I compute the extraction of some data of interest
out of a larger data-set, I wish not to repeat that operation
over and over each time I run this notebook.
I tested it and de-serialization is pretty fast
(4000000 3 keys dictionaries de-serialized in 4 seconds)
'''
import pickle # for each recording in Listenbrainz 2022 month
import csv # for reading -and creating a smaller "version" of- listenbrainz_msid_mapping.csv

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Read the jsonlines data from ListenBrainzData/listenbrainz- monthly data files


In [3]:
# Please input here the absolute path of the ListenBrainz 2022 data January file. The script will compute the rest of the months data file paths
month_1_listenBrainz_data_file_path = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/listenbrainz-2022/1.listens'
!head '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/listenbrainz-2022/1.listens'

# create a list where we will store the paths of the 2022 ListenBrainz data files (12 files -> 12 items in the list)
listenBrainz_data_from_2022_files_paths = []
listenBrainz_data_from_2022_files_paths.append(month_1_listenBrainz_data_file_path)
# delete '1.listens' from month_1_data_file
listenBrainzData_parentFolder = listenBrainz_data_from_2022_files_paths[0][:-9]
for month_n in range(1,12):
  listenBrainz_data_from_2022_files_paths.append(str(str(listenBrainzData_parentFolder) + str(month_n+1) + '.listens'))

{"user_id":17240,"user_name":"Winterbay","timestamp":1640995200,"track_metadata":{"artist_name":"Tiken Jah Fakoly","release_name":"Coup De Gueule","additional_info":{"artist_msid":"f03fa31b-a428-47e2-8dcb-bceec9ca1221","release_msid":"238bd863-a293-4718-a66b-093ea54bf8f3","listening_from":"lastfm","recording_msid":"f216e2fb-784d-470a-81b2-6e27cd532204","lastfm_artist_mbid":"edef3cfa-4e5e-4d64-8bd8-20f9dc1d8cad","lastfm_release_mbid":"9dc7fe6a-3fa4-4461-8975-ecb7218b39a3"},"track_name":"Alou Maye"},"recording_msid":"f216e2fb-784d-470a-81b2-6e27cd532204"}
{"user_id":16930,"user_name":"kazoo","timestamp":1640995200,"track_metadata":{"artist_name":"Fraktus","release_name":"Millennium Edition","additional_info":{"artist_msid":"afe0b08d-f47d-4adf-bd78-3e95ca276f7c","tracknumber":11,"release_msid":"9cd2285c-4860-4e83-b8bb-5fef7328b224","recording_msid":"7a490913-f4d5-40a8-b34a-02166fbc511e"},"track_name":"Computerliebe"},"recording_msid":"7a490913-f4d5-40a8-b34a-02166fbc511e"}
{"user_id":1513

In [ ]:
# Run this cell only for test purposes
batch_size = 100000

In [4]:
user_info_and_messyBrainz_ids_dictList = []
user_info_and_messyBrainz_ids_dict = {}

Here I test the speed of 3 different approaches in JSON line to vocabulary (filtering only the keys we need) conversion;

- Dictionary comprehension
- dict() constructor
- filter method

(https://www.geeksforgeeks.org/python-extract-specific-keys-from-dictionary/)

In [ ]:
# Run this cell only for test purposes
##### "MASKED" DICTIONARY COMPREHENSION

user_info_and_messyBrainz_ids_dictList = []
user_info_and_messyBrainz_ids_dict = {}
line_counter = 0
start = time.monotonic()
with open(listenBrainz_data_from_2022_files_paths[0]) as json_lines_file:
  for json_line in json_lines_file:
    if line_counter >= batch_size:
      break
    user_info_and_messyBrainz_ids_dict = json.loads(json_line)
    user_info_and_messyBrainz_ids_dictList.append({key: user_info_and_messyBrainz_ids_dict[key] for key in user_info_and_messyBrainz_ids_dict.keys() & {'user_id', 'recording_msid'}})
    line_counter += 1
end = time.monotonic()

print(f'{len(user_info_and_messyBrainz_ids_dictList)} json lines processed in {end - start} seconds.')

100000 json lines processed in 1.4730764449996059 seconds.


In [ ]:
# Run this cell only for test purposes
##### DICTIONARY CONSTRUCTOR

user_info_and_messyBrainz_ids_dictList = []
user_info_and_messyBrainz_ids_dict = {}
line_counter = 0
start = time.monotonic()
with open(listenBrainz_data_from_2022_files_paths[0]) as json_lines_file:
  for json_line in json_lines_file:
    if line_counter >= batch_size:
      break
    user_info_and_messyBrainz_ids_dict = json.loads(json_line)
    user_info_and_messyBrainz_ids_dictList.append(dict((k, user_info_and_messyBrainz_ids_dict[k]) for k in ['user_id', 'recording_msid'] if k in user_info_and_messyBrainz_ids_dict))
    line_counter += 1
end = time.monotonic()

print(f'{len(user_info_and_messyBrainz_ids_dictList)} json lines processed in {end - start} seconds.')

100000 json lines processed in 2.5338072109998393 seconds.


In [ ]:
# Run this cell only for test purposes
##### FILTER METHOD

user_info_and_messyBrainz_ids_dictList = []
user_info_and_messyBrainz_ids_dict = {}
line_counter = 0
start = time.monotonic()
with open(listenBrainz_data_from_2022_files_paths[0]) as json_lines_file:
  for json_line in json_lines_file:
    if line_counter >= batch_size:
      break
    user_info_and_messyBrainz_ids_dictList.append(dict(filter(lambda item: item[0] in ['user_id', 'recording_msid'], json.loads(json_line).items())))
    line_counter += 1
end = time.monotonic()

print(f'{len(user_info_and_messyBrainz_ids_dictList)} json lines processed in {end - start} seconds.')

100000 json lines processed in 1.43607589200019 seconds.


I realized that testing with a small batch size tends to noisy (very variate, but also very similar to each other) results. As the batch size increases, results are still non-deterministic but tend to stabilize (that is, the difference between the speeds tends to be more noticeable in terms of results).

Dictionary comprehension seems to be the fastest in average over 4 different tests (the same 3 tests repeated 4 times).

# Perform the actual retrieval of user id and MessyBrainz ids (with no batch and for each month separately)

In [5]:
monthly_data_pickleFiles_folderPath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1'

Store the filtered data of interest from each month of the year 2022 in separate .pickle files

In [ ]:
user_info_and_messyBrainz_ids_dict = {}
line_counter = 0
for month_n in range(12):
  start = time.monotonic()
  with open(listenBrainz_data_from_2022_files_paths[month_n]) as json_lines_file:
    user_info_and_messyBrainz_ids_dictList = [] # empty the list of dictionaries, otherwise we will run out of RAM (previous months data have already been stored in a pickle file)
    for json_line in json_lines_file:
      user_info_and_messyBrainz_ids_dict = json.loads(json_line)
      user_info_and_messyBrainz_ids_dictList.append({key: user_info_and_messyBrainz_ids_dict[key] for key in user_info_and_messyBrainz_ids_dict.keys() & {'user_id', 'user_name', 'recording_msid'}})
  end = time.monotonic()
  # pickle dump
  pickle_file_path = str(monthly_data_pickleFiles_folderPath + '/userInfo_MessyBrainzIDs_month_' + str(month_n+1) + '.p')
  pickle.dump(user_info_and_messyBrainz_ids_dictList, open(pickle_file_path, "wb" ) )
  print(f'{len(user_info_and_messyBrainz_ids_dictList)} json lines processed in {end - start} seconds for month n. {month_n+1}.')
  print(f'These data have been serialized into a pickle file at {pickle_file_path}')

4339352 json lines processed in 85.22099045599953 seconds for month n. 1.
These data have been serialized into a pickle file at /content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/userInfo_MessyBrainzIDs_month_1.p
3980638 json lines processed in 73.60381743200014 seconds for month n. 2.
These data have been serialized into a pickle file at /content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/userInfo_MessyBrainzIDs_month_2.p
3948158 json lines processed in 71.70004340000014 seconds for month n. 3.
These data have been serialized into a pickle file at /content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/userInfo_MessyBrainzIDs_month_3.p
4177942 json lines processed in 83.33782375299961 seconds for month n. 4.
These data have been serialized into a pickle file at /content/gdrive/MyDr

Try to de-serialize back the pickle files (DON'T RUN)

In [ ]:
# Run this cell only for test purposes
month_n = 0
pickle_file_path = str(monthly_data_pickleFiles_folderPath + '/userInfo_MessyBrainzIDs_month_' + str(month_n+1) + '.p')
start = time.monotonic()
month1_data = pickle.load(open(pickle_file_path, "rb") )
end = time.monotonic()

print(f'Pickle file deserialized in {end - start} seconds ({len(month1_data)} dictionaries) for month n. {month_n+1}.')

Pickle file deserialized in 4.7836960569998155 seconds (4339352 dictionaries) for month n. 1.


# A fast way to map MessyBrainz ids to MusicBrainz ids could (hopefully) be to just create a SET (a list with unique occurrences, no repetitions, and low index complexity time) out of the list of MessyBrainz ids (which can be computed out of the whole 2022 data, rather than month by month).

In [ ]:
unique_MessyBrainz_ids = set()
uinque_User_ids = set()
user_IDs_to_num_MessyBrainzIDs_listenings_dict = dict()

for month_n in range(12):
  pickle_file_path = str(monthly_data_pickleFiles_folderPath + '/userInfo_MessyBrainzIDs_month_' + str(month_n+1) + '.p')
  start = int(time.monotonic()) # I don't need decimal precision
  thisMonth_userInfo_MessyBrainzIds = pickle.load(open(pickle_file_path, "rb") )
  end = int(time.monotonic())
  print(f'Pickle file for month n. {month_n+1} deserialized in {end - start} seconds ({len(thisMonth_userInfo_MessyBrainzIds)} dictionaries).')
  start = int(time.monotonic()) 
  for month_data in thisMonth_userInfo_MessyBrainzIds: # list of dictionaries
    unique_MessyBrainz_ids.add(month_data['recording_msid'])
    uinque_User_ids.add(month_data['user_id'])
    if month_data['user_id'] in user_IDs_to_num_MessyBrainzIDs_listenings_dict:
      # print('Appending new values in existing key in user_IDs_to_num_MessyBrainzIDs_listenings_dict')
      user_IDs_to_num_MessyBrainzIDs_listenings_dict[month_data['user_id']].append(month_data['recording_msid'])
    else:
      # print('Inserting new key in user_IDs_to_num_MessyBrainzIDs_listenings_dict')
      user_IDs_to_num_MessyBrainzIDs_listenings_dict[month_data['user_id']] = [month_data['recording_msid']]
  end = int(time.monotonic())
  print(f'Finished extracting unique MessyBrainz ids and unique User ids for month n. {month_n+1} in {end-start} seconds.')
print(f'Finished extracting unique MessyBrainz ids and User ids for the whole 2022 year.')
print(f'There are a total of {len(unique_MessyBrainz_ids)} unique MessyBrainz ids.')
print(f'There are a total of {len(uinque_User_ids)} unique User ids ids.')

Pickle file for month n. 1 deserialized in 7 seconds (4339352 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 1 in 6 seconds.
Pickle file for month n. 2 deserialized in 9 seconds (3980638 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 2 in 6 seconds.
Pickle file for month n. 3 deserialized in 5 seconds (3948158 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 3 in 7 seconds.
Pickle file for month n. 4 deserialized in 4 seconds (4177942 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 4 in 9 seconds.
Pickle file for month n. 5 deserialized in 3 seconds (4403964 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 5 in 10 seconds.
Pickle file for month n. 6 deserialized in 2 seconds (4154861 dictionaries).
Finished extracting unique MessyBrainz ids and unique User ids for month n. 6

In [ ]:
print(type(user_IDs_to_num_MessyBrainzIDs_listenings_dict[17240]))

<class 'list'>


In [ ]:
# pickle dump
unique_MessyBrainz_ids_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/unique_MessyBrainz_ids.p')
pickle.dump(unique_MessyBrainz_ids, open(unique_MessyBrainz_ids_pickle_file_path, "wb" ) )

uinque_User_ids_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/uinque_User_ids.p')
pickle.dump(uinque_User_ids, open(uinque_User_ids_pickle_file_path, "wb" ) )

In [ ]:
user_IDs_to_num_MessyBrainzIDs_listenings_dict_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/user_IDs_to_num_MessyBrainzIDs_listenings_dict.p')
pickle.dump(user_IDs_to_num_MessyBrainzIDs_listenings_dict, open(user_IDs_to_num_MessyBrainzIDs_listenings_dict_pickle_file_path, "wb" ) )

# RESTART RUNTIME
Simplify user_IDs_to_num_MessyBrainzIDs_listenings_dict, so that for each MessyBrainzID we only have 1 occurence of it, and also the counts of its number of listenings

In [ ]:
user_IDs_to_num_MessyBrainzIDs_listenings_dict_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/user_IDs_to_num_MessyBrainzIDs_listenings_dict.p')
user_IDs_to_num_MessyBrainzIDs_listenings_dict = pickle.load(open(user_IDs_to_num_MessyBrainzIDs_listenings_dict_pickle_file_path, "rb"))
for user_id in user_IDs_to_num_MessyBrainzIDs_listenings_dict:
  list_of_MessyBrainzIDs_listened_byThisUser = user_IDs_to_num_MessyBrainzIDs_listenings_dict[user_id]
  list_of_Unique_MessyBrainzIDs_listened_byThisUser, counts = numpy.unique(list_of_MessyBrainzIDs_listened_byThisUser, return_counts=True)
  user_IDs_to_num_MessyBrainzIDs_listenings_dict[user_id] = list(zip(list_of_Unique_MessyBrainzIDs_listened_byThisUser, counts))

pickle.dump(user_IDs_to_num_MessyBrainzIDs_listenings_dict, open(user_IDs_to_num_MessyBrainzIDs_listenings_dict_pickle_file_path, "wb" ) )

Count how many occurrences of each MessyBrainzID there are for each user_id in user_IDs_to_num_MessyBrainzIDs_listenings_dict.
Then store everything in a .csv file.

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/user_IDs_to_num_MessyBrainzIDs_listenings_mapping.csv', "w") as user_IDs_to_num_MessyBrainzIDs_listenings_mapping_file:
  fieldnames = ['user_id']
  fieldnames = fieldnames + list(unique_MessyBrainz_ids)
  csvWriter = csv.DictWriter(user_IDs_to_num_MessyBrainzIDs_listenings_mapping_file, fieldnames=fieldnames)
  csvWriter.writeheader()
  start = int(time.monotonic())
  for user_id in user_IDs_to_num_MessyBrainzIDs_listenings_dict:
    # print(user_id)
    list_of_MessyBrainzIDs_listened_byThisUser = user_IDs_to_num_MessyBrainzIDs_listenings_dict[user_id]
    list_of_Unique_MessyBrainzIDs_listened_byThisUser, counts = numpy.unique(list_of_MessyBrainzIDs_listened_byThisUser, return_counts=True)
    print(list_of_Unique_MessyBrainzIDs_listened_byThisUser)
    print(counts)
    for messyBrainzID in list_of_Unique_MessyBrainzIDs_listened_byThisUser:
      csvWriter.writerow({'user_id': user_id, messyBrainzID: counts})
  end = int(time.monotonic()) # end of csvWriter iteration
  '''
  print(f'Finished creating new filtered .csv file with only the MessyBrainzIDs <-> ListenBrainzIDs mappings for which there has been a listening')
  print(f'It took {end - start} seconds, and {num_matches} matches (listenings) were found.')
  print(f'A set of {len(unique_MusicBrainz_Recordings_ids)} unique MusicBrainz recording IDs has been created, this is the real number of songs that have been listened to in 2022 by all users.')
    '''

['000061d6-0ab0-43a2-b6b7-ec1c6e35da68'
 '0000cf6a-1829-4724-a8fa-f52ae32aa117'
 '0001ca3a-ae6c-4d0c-b81a-35ff664dea8a' ...
 'fffdfd52-c9af-451f-bf24-63d9b3b5a3e3'
 'fffe24a9-265d-4cd3-98d0-5f5ba0490dc7'
 'fffffcc6-6e62-4393-bc81-bb7a4ac1b7b2']
[2 1 1 ... 1 1 2]


KeyboardInterrupt: ignored

# Get only NEEDED DATA from listenbrainz_msid_mapping.csv and create a new (much smaller) .csv with those data

In [ ]:
# This file is huge (around 5.50 GB)
msid_mbid_mapping_filePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/listenbrainz_msid_mapping.csv'
!head -20 '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/listenbrainz_msid_mapping.csv'

recording_msid,recording_mbid,match_type
00000737-3a59-4499-b30a-31fe2464555d,1fe669c9-5a2b-4dcb-9e95-77480d1e732e,exact_match
000013b3-dbb4-43a0-8fd4-ca92ff5ed033,c5bfd98d-ccde-4cf3-8abb-63fad1b6065a,exact_match
00002714-6f74-409d-9fa4-441c8dfb195f,c6acc112-3df7-4716-b5b6-953b5e93743f,exact_match
00003a81-2a6c-4d6c-ad43-990c0806458b,007770e2-90c5-49f2-b894-690db7ebea40,exact_match
00005660-7eb0-4592-a74b-14f3de9cc4cb,67bcde07-bfb1-4b30-88ba-6b995ec04123,exact_match
00006a3b-babd-4bb0-89b3-e2835aba6425,7d52ec8a-3d84-4e3d-8463-657d749ba4d4,exact_match
0000825d-b547-43a7-b294-948e8e472766,99687951-9b5d-4918-b08e-39921d736d68,high_quality
00008dc7-e09c-451e-85a0-7ce1db8fbe19,e44e1b73-9750-4712-a1c3-f57ae2a84f5a,med_quality
00009fc5-4f28-4020-b02c-966d6c5e4202,1dd0bc8f-f8c3-4658-b397-ec9ec87e618f,exact_match
0000b284-1f92-404f-97ab-12c89356de22,9d1601c9-dcd3-4c67-b780-933f4604a8cd,exact_match
0000bab1-b30b-457d-b96f-c2b17da61ee7,42355d8b-f897-4e61-99c1-df3ea52119e2,exact_match
0000c3e1-ae5

For each month of user_info/MessyBrainz_recording_id data, extract from listenbrainz_msid_mbid_mapping_filePath ONLY the mappings for which there has been a listening.
The third row of listenbrainz_msid_mbid_mapping_filePath can always be discarded.

Create also a set of unique MusicBrainz recordings IDs, will come useful later when mapping each MusicBrainz recording to an Artist credit ID

In [ ]:
unique_MusicBrainz_Recordings_ids = set()
num_matches = 0
with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/msid_mbid_filtered_mapping_file.csv', "w") as msid_mbid_filtered_mapping_file:
  fieldnames = ['recording_msid', 'recording_mbid']
  csvWriter = csv.DictWriter(msid_mbid_filtered_mapping_file, fieldnames=fieldnames)
  csvWriter.writeheader()
  with open(msid_mbid_mapping_filePath) as msid_mbid_mapping_file:
    csvReader = csv.reader(msid_mbid_mapping_file)
    start = int(time.monotonic())
    for csvLine in csvReader:
      # first column is MessyBrainz id
      if csvLine[0] in unique_MessyBrainz_ids:
        # If a recording_msid has actually been listened to
        # add it to the new .csv file
        # print((csvLine[0], csvLine[1]))
        csvWriter.writerow({'recording_msid': csvLine[0], 'recording_mbid': csvLine[1]})
        unique_MusicBrainz_Recordings_ids.add(csvLine[1])
        num_matches += 1
    end = int(time.monotonic()) # end of csvReader iteration
    print(f'Finished creating new filtered .csv file with only the MessyBrainzIDs <-> ListenBrainzIDs mappings for which there has been a listening')
    print(f'It took {end - start} seconds, and {num_matches} matches (listenings) were found.')
    print(f'A set of {len(unique_MusicBrainz_Recordings_ids)} unique MusicBrainz recording IDs has been created, this is the real number of songs that have been listened to in 2022 by all users.')

Finished creating new filtered .csv file with only the MessyBrainzIDs <-> ListenBrainzIDs mappings for which there has been a listening
It took 236 seconds, and 11291747 matches (listenings) were found.


The number of matches between MessyBrainzIDs and ListenBrainzIDs should be equal to the number of unique MessyBrainzIDs previously found.
It is equal.

In [ ]:
# pickle dump
unique_MusicBrainz_Recordings_ids_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/unique_MusicBrainz_Recordings_ids.p')
pickle.dump(unique_MusicBrainz_Recordings_ids, open(unique_MusicBrainz_Recordings_ids_pickle_file_path, "wb" ) )

# Get only NEEDED DATA from canonical_musicbrainz_data.csv and create a new (much smaller) .csv with those data

In [ ]:
# This file is huge (around 5 GB)
canonical_musicBrainz_data_filePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/metabrainz-metadata-dump-20230117-172210/metabrainz/canonical_musicbrainz_data.csv'
!head -20 '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/metabrainz-metadata-dump-20230117-172210/metabrainz/canonical_musicbrainz_data.csv'

id,artist_credit_id,artist_mbids,artist_credit_name,release_mbid,release_name,recording_mbid,recording_name,combined_lookup,score,year
28939355,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,5f5f649f-1938-4a3e-a879-a95693a99a71,Heavenly Flow,michieoneheavenlyflow,371181,2006
28939356,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,6ee381af-e9b4-46c4-a4f0-541dce2f03ea,Party,michieoneparty,371181,2006
28939357,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,8b371ea0-dee1-4fbf-bacd-3aa87a4aef13,Free Like Jah,michieonefreelikejah,371181,2006
28939358,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,9d904f0f-314b-4089-b151-d81e0857c431,People,michieonepeople,371181,2006
28939359,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-

Extract from canonical_musicBrainz_data_filePath ONLY the credits_IDs (probably redundant) and artists IDs for which there has been a listening.
The other rows and columns of canonical_musicBrainz_data_filePath can always be discarded.

Also create a set of unique artist IDs, it will be used later when filtering a .csv file with artistID <-> artist name mapping.

In [ ]:
unique_MusicBrainz_Artist_ids = set()
num_matches = 0
with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/canonical_musicBrainz_data_filtered.csv', "w") as canonical_musicBrainz_data_filtered:
  fieldnames = ['recording_mbid', 'artist_credit_id', 'artist_mbids']
  csvWriter = csv.DictWriter(canonical_musicBrainz_data_filtered, fieldnames=fieldnames)
  csvWriter.writeheader()
  with open(canonical_musicBrainz_data_filePath) as canonical_musicBrainz_data_file:
    csvReader = csv.reader(canonical_musicBrainz_data_file)
    start = int(time.monotonic())
    for csvLine in csvReader:
      if csvLine[6] in unique_MusicBrainz_Recordings_ids:
        csvWriter.writerow({'recording_mbid': csvLine[6], 'artist_credit_id': csvLine[1], 'artist_mbids': csvLine[2]})
        # MULTIPLE ARTISTS ARE CONSIDERED AS 1 NEW SEPARATE ARTIST
        for artist_mbids in [csvLine[2]]: # convert set to a list
          # artist_mbids is a string, remove brackets and separate by commas
          artist_mbids = artist_mbids.replace('{','')
          artist_mbids = artist_mbids.replace('}','')
          artist_mbids = artist_mbids.split(',')
          for artist_mbid in artist_mbids:
            # print(str(artist_mbid))
            unique_MusicBrainz_Artist_ids.add(str(artist_mbid))
        num_matches += 1
    end = int(time.monotonic()) # end of csvReader iteration
    print(f'Finished creating new filtered .csv file with only the MusicBrainzIDs <-> artist_credit_id <-> artist_mbids mappings for which there has been a listening')
    print(f'It took {end - start} seconds, and {num_matches} records were found.')
    print(f'A set of {len(unique_MusicBrainz_Artist_ids)} unique MusicBrainz artist IDs has been created.')

Streaming output truncated to the last 5000 lines.
b945dbec-14ac-45a6-bd52-bee3ad5c7134
b945dbec-14ac-45a6-bd52-bee3ad5c7134
5b0ffc70-5573-478a-9b68-342da5089ec9
320f4142-eb12-42b7-8395-9545f4239cb4
c6fab48d-6707-4e25-b803-98a0b623410f
c6fab48d-6707-4e25-b803-98a0b623410f
c6fab48d-6707-4e25-b803-98a0b623410f
c6fab48d-6707-4e25-b803-98a0b623410f
c6fab48d-6707-4e25-b803-98a0b623410f
c0a1179b-b14a-4d68-a3c1-1fdab16ed602
75a72702-a5ef-4513-bca5-c5b944903546
7c158ea8-c0aa-410e-bdc1-20bba9759577
592da4c4-9618-4a1a-a944-e60c05c39037
4a8167f6-d207-4571-9f52-16f849fb721c
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-0d85b8fdeb75
1e34b1b8-05a2-4672-be10-

Dump unique_MusicBrainz_Artist_ids into a pickle file

In [ ]:
# pickle dump
unique_MusicBrainz_Artist_ids_pickle_file_path = str('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/unique_MusicBrainz_Artist_ids.p')
pickle.dump(unique_MusicBrainz_Artist_ids, open(unique_MusicBrainz_Artist_ids_pickle_file_path, "wb" ) )

# Get only NEEDED DATA from musicbrainz_artist_mbid_name.csv and create a new (much smaller) .csv with those data

In [ ]:
musicbrainz_artist_mbid_name_filePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/musicbrainz_artist_mbid_name.csv'
!head -20 '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/ListenBrainzData/musicbrainz_artist_mbid_name.csv'

mbid,name
fadeb38c-833f-40bc-9d8c-a6383b38b1be,Доктор Сатана
49add228-eac5-4de8-836c-d75cde7369c3,Pete Moutso
c112a400-af49-4665-8bba-741531d962a1,Zachary
ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b,The Silhouettes
7b4a548e-a01a-49b7-82e7-b49efeb9732c,Aric Leavitt
60aca66f-e91a-4cb5-9308-b6e293cd833e,Fonograff
3e1bd546-d2a7-49cb-b38d-d70904a1d719,Al Street
df120895-f6c6-4a66-b9cf-73350f0beb61,Love .45
c14f8d3f-ee81-416f-800f-8eff7e77a2e1,Sintellect
b68a3969-319a-462f-942b-cd35581414fc,Evie Tamala
2c8ae2e0-3934-440e-81f5-2ec7fd0d7899,Jean-Pierre Martin
ac63d693-7b24-4258-a3db-09743b1b4269,Deejay One
4c4b7c6f-9285-4d6a-bc10-e5c9e08045f8,wecamewithbrokenteeth
055f435f-dba6-4156-9050-6ac41113e45f,The Blackbelt Band
ab1b631b-9896-4433-bef9-7868bf8a42f3,Giant Tomo
66de1369-f9eb-43cb-ae4f-88582a47a624,Elvin Jones & Jimmy Garrison Sextet
1fbb9556-b647-498a-a8ed-d3b5e8d7f85c,Tobias Lorsbach
e6895f6e-f636-4ff6-b406-f5ddaf6cb243,Diskobitch
4eee2c60-c2c8-4b33-b14f-0eed4bf4d11a,Seanews


In [ ]:
num_matches = 0
with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/musicbrainz_artist_mbid_name_filtered.csv', "w") as musicbrainz_artist_mbid_name_filtered:
  fieldnames = ['mbid', 'name']
  csvWriter = csv.DictWriter(musicbrainz_artist_mbid_name_filtered, fieldnames=fieldnames)
  csvWriter.writeheader()
  with open(musicbrainz_artist_mbid_name_filePath) as musicbrainz_artist_mbid_name_file:
    csvReader = csv.reader(musicbrainz_artist_mbid_name_file)
    start = int(time.monotonic())
    for csvLine in csvReader:
      if csvLine[0] in unique_MusicBrainz_Artist_ids:
         csvWriter.writerow({'mbid': csvLine[0], 'name': csvLine[1]})
         num_matches += 1
    end = int(time.monotonic()) # end of csvReader iteration
    print(f'Finished creating new filtered .csv file with only the MusicBrainz artist IDs <-> artist names mappings for which there has been a listening')
    print(f'It took {end - start} seconds, and {num_matches} records were found.')

Finished creating new filtered .csv file with only the MusicBrainz artist IDs <-> artist names mappings for which there has been a listening
It took 6 seconds, and 353007 records were found.


# We need to convert the ListenBrainz data dump to a matrix containing user ids, artist ids, and play counts (how many times someone listened to a given artist in 2022)

Create a .csv file containing the number of listening for each MessyBrainzID for each user_id.

Yes, 'apple' is in this set
Yes, 'apple' is in this set
